In [ ]:
from ultralytics import YOLO
import cv2
import numpy as np
import torchreid
from scipy.spatial.distance import cosine
from deep_sort_realtime.deepsort_tracker import DeepSort
from sklearn.metrics.pairwise import cosine_similarity
import torch
import csv
import os
from datetime import datetime

# Store appearance times
appearance_times = {}

csv_file = "timelines.csv"

# Create CSV with header if not exists
if not os.path.exists(csv_file):
    with open(csv_file, mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Person ID", "Appearance Time"])

def log_appearance(person_id):
    """Record first appearance and save to CSV if new"""
    if person_id not in appearance_times:
        appearance_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        appearance_times[person_id] = appearance_time
        print(f" Person {person_id} appeared at {appearance_time}")

        # Append to CSV
        with open(csv_file, mode="a", newline="") as file:
            writer = csv.writer(file)
            writer.writerow([person_id, appearance_time])


# Initializing the models
yolo_model = YOLO("yolo11n.pt")
extractor = torchreid.utils.FeatureExtractor(
    model_name='osnet_x1_0',
    device='cpu'
)

# Initializing the DeepSORT tracker
tracker = DeepSort(max_age=20, n_init=7)

# Gallery to store unique person embeddings
gallery = {}
next_person_id = 0
person_id_map = {}

cap = cv2.VideoCapture(0)
min_conf = 0.5
while True:
    detections = []
    check, frame = cap.read()
    if not check:
        break
    
    # YOLO detection
    result = yolo_model(frame, verbose=False)[0]
    for box in result.boxes:
        if box.cls[0] != 0 or box.conf[0] < min_conf:
            continue
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf[0])
        detections.append(([x1, y1, x2 - x1, y2 - y1], conf, 'person'))

    # DeepSORT tracking
    tracks = tracker.update_tracks(detections, frame=frame)
    
    for track in tracks:
        if not track.is_confirmed():
            continue
        
        track_id = track.track_id
        l, t, r, b = map(int, track.to_ltrb())
        log_appearance(track_id)
        
        # Check if the track ID is in our person_id map
        if track_id not in person_id_map:
            # New track ID, perform ReID check
            cropped = frame[t:b, l:r]
            if cropped.size == 0:
                continue
            
            resized = cv2.resize(cropped, (128, 256))
            feature = extractor([resized])[0].cpu()

            best_match_id = None
            best_similarity_score = 0.0

            # Compare with gallery
            if gallery:
                gallery_embeddings = np.array([data['embedding'].numpy() for data in gallery.values()])
                similarity_scores = cosine_similarity(
                    feature.unsqueeze(0).numpy(),
                    gallery_embeddings
                )[0]
                
                # max_score_idx = np.argmax(similarity_scores)
                # best_similarity_score = similarity_scores[max_score_idx]
                similarity_scores = np.sort(similarity_scores)
                best_similarity_score = np.mean(similarity_scores[-2:-1])
                
                # Minimum similarity threshold
                if best_similarity_score > 0.6:
                    best_match_id = list(gallery.keys())[-1]

            if best_match_id is not None:
                # Match found, link the new track_id to the existing person_id
                person_id_map[track_id] = best_match_id
                print(f"Track {track_id} matched with known person ID {best_match_id} (score={best_similarity_score:.2f})")
            else:
                # No match found, create a new person ID
                current_person_id = next_person_id
                next_person_id += 1
                
                gallery[current_person_id] = {'embedding': feature}
                person_id_map[track_id] = current_person_id
                print(f"Track {track_id} assigned new person ID {current_person_id}")

        # Get the person_id from the map
        person_id = person_id_map[track_id]
        
        # Draw
        cv2.rectangle(frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(frame, f"Person {person_id}", (l, t - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    cv2.imshow("ReID + Tracking", frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

/home/ritwik/Ritwik/Programming/ml/CCTV/deep-person-reid/torchreid/models/osnet.py:482: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(cached_file)
/h

Successfully loaded imagenet pretrained weights from "/home/ritwik/.cache/torch/checkpoints/osnet_ibn_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']
Model: osnet_ibn_x1_0
- params: 2,194,640
- flops: 978,878,352


[ WARN:0@369.305] global cap_gstreamer.cpp:1173 isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


Track 1 assigned NEW Person 0
Track 4 assigned NEW Person 1
Track 5 matched with Person 0 (score=0.82)
Track 7 assigned NEW Person 2
Track 10 assigned NEW Person 3
Track 9 assigned NEW Person 4
Track 11 assigned NEW Person 5
Track 14 assigned NEW Person 6
Track 15 assigned NEW Person 7
Track 16 assigned NEW Person 8
Track 17 assigned NEW Person 9
Track 19 matched with Person 9 (score=0.83)


## 